In [1]:
cd ../../../../Apps/Python/bolsao-api

C:\Users\luisr\Desktop\Repositories\Apps\Python\bolsao-api


In [4]:
import pandas as pd, json
from modules.polygons import Polygon
from modules.geojson_conversion import polygons_to_df

# ---
# urls monitoring parameters

# api_root = 'https://bolsao-api-j2fefywbia-rj.a.run.app'
api_root = 'http://127.0.0.1:5000'

# Obs: add 'prefix' param if the url returns point data. if the url data is filtered, put the 'prefix' param in the 'filter' object inside the 'filters' param. this will generate the 'col_status' and 'col_count' fields in 'polygon.status()' method and also the 'col_status_count' field in the 'polygon.general_status()' method

urls_history = [{
    'url': '/comando/bq/open',
    'id': 'id_evento',
    'coords': ['longitude', 'latitude'],
    'prefix': 'comando',
    'filters': [{
        'params': {'id_pop': 33},
        'prefix': 'lâmina'
    }, {
        'params': {'id_pop': 5},
        'prefix': 'bolsão'
    }, {
        'params': {'id_pop': 31},
        'prefix': 'alagamento'
    }, {
        'params': {'id_pop': 6},
        'prefix': 'alagamento_enchente'
    }, {
        'params': {'id_pop': 32},
        'prefix': 'enchente'
    }, {
        'params': {'id_pop': 16},
        'prefix': 'vazamento'
    }, {
        'params': {'id_pop': 30},
        'prefix': 'sirene'
    }]
}]

for url in urls_history: url['url'] = api_root + url['url']

# ---
# logic states parameters

logic_history = [{
    'name': 'NORMALIDADE',
    'fields': []
}, {
    'name': 'ATENCAO',
    'fields': [] # ['label', 'waze_flood_status']
}, {
    'name': 'ALERTA',
    'fields': ['lâmina_status']    
}, {
    'name': 'CRITICO',
    'fields': ['bolsão_status', 'alagamento_status', 'alagamento_enchente_status', 'enchente_status']
}]

## INITIALIZE POLYGONS ---

# ---
# Load polygons dataset

polygons_df = pd.read_csv('static/clusters/clusters_micro.csv')

city_polygon_df = polygons_df[polygons_df['sublabel'] == -1]
polygons_df = polygons_df[polygons_df['sublabel'] != -1]

path_ipp = 'static/ipp/areas_alagamento_Polygon.geojson'
ipp_df = polygons_to_df(json.loads(open(path_ipp, 'rb').read()))
ipp_df = ipp_df[ipp_df['soma_peso'].isin([6, 4])]

# ---
# columns parameters

geometry_cols = ['lng_min', 'lng_max', 'lat_min', 'lat_max']

clusters_drop = [
    'radius', 'horizontal_perimeter', 'density_circle', 'density_box',
    'vertical_perimeter', 'lng_center', 'title', 'lat_center',
    'area_circle'
]

prob_drop = ['_id', 'cluster', 'range']
comando_drop = []
waze_drop = []

drop_from_status = clusters_drop + prob_drop + comando_drop + waze_drop

# ---
# Instantiate Polygon class objects

polygons_history = Polygon(
    polygons_df, poly_id='sublabel', base_id='cluster_id', urls=urls_history, logic=logic_history,
    drop_from_status=drop_from_status, geometry_cols=geometry_cols
)

city_polygon_history = Polygon(
    city_polygon_df, poly_id='sublabel', base_id='cluster_id', urls=urls_history, logic=logic_history,
    drop_from_status=drop_from_status, geometry_cols=geometry_cols
)

polygons_ipp_history = Polygon(
    ipp_df, poly_id='objectid', base_id='cluster_id', urls=urls_history, logic=logic_history,
    drop_from_status=[], geometry_cols='coordinates'
)

### Example Usage

#### Polygon object normal behavior

In [26]:
display(polygons.general_status(drop_geometry=False).head())

display(polygons.status().head())

,timestamp,status_code,status_name,NORMALIDADE,ATENCAO,ALERTA,CRITICO,label_status,acumulado_chuva_15_min - 1_status,acumulado_chuva_15_min - 2_status,...,sirene_status_count,waze_flood_ids,lâmina_ids,bolsão_ids,alagamento_ids,alagamento_enchente_ids,enchente_ids,vazamento_ids,sirene_ids,coordinates
0,timestamp 2023-02-12 17:01:44 timestamp ...,0,NORMALIDADE,79,0,0,0,0,0,0,...,0,[],[],[],[],[],[],[],[],"[[[[-43.3128293, -23.0071335], [-43.3128293, -..."


,status_code,status_name,timestamp,cluster_id,main_neighborhood,main_route,main_street_number_range,lat_centroid,lng_centroid,label_count,...,alagamento_enchente_ids,enchente_count,enchente_status,enchente_ids,vazamento_count,vazamento_status,vazamento_ids,sirene_count,sirene_status,sirene_ids
0,0,NORMALIDADE,2023-02-12 18:38:28,0,Barra da Tijuca,Avenida Armando Lombardi,3098 - 67,-23.006631,-43.310232,114,...,[],0,0,[],0,0,[],0,0,[]
1,0,NORMALIDADE,2023-02-12 18:38:28,1,Catete,Rua do Catete,228 - 139,-22.926423,-43.176842,99,...,[],0,0,[],0,0,[],0,0,[]
2,0,NORMALIDADE,2023-02-12 18:38:28,2,Copacabana,Rua Tonelero,236 - 9,-22.966793,-43.185999,43,...,[],0,0,[],0,0,[],0,0,[]
3,0,NORMALIDADE,2023-02-12 18:38:28,3,Ipanema,Avenida Epitácio Pessoa,1910 - 1602,-22.979735,-43.202540,36,...,[],0,0,[],0,0,[],0,0,[]
4,0,NORMALIDADE,2023-02-12 18:38:28,4,Barra da Tijuca,Avenida Ministro Ivan Lins,1770 - 11,-23.012736,-43.298339,36,...,[],0,0,[],0,0,[],0,0,[]


#### Update url to monitor open events at given time window

In [27]:
polygons.urls[0]['url'] = api_root + '/comando/bq/history' + '?start=2023-01-07 00:00:00&end=2023-01-07 23:59:59'

display(polygons.status().head())
display(polygons.general_status().head())

,status_code,status_name,timestamp,index,main_neighborhood,main_route,main_street_number_range,lat_centroid,lng_centroid,label_count,...,alagamento_enchente_ids,enchente_count,enchente_status,enchente_ids,vazamento_count,vazamento_status,vazamento_ids,sirene_count,sirene_status,sirene_ids
0,0,NORMALIDADE,2023-02-12 18:38:45,0.0,Barra da Tijuca,Avenida Armando Lombardi,3098 - 67,-23.006631,-43.310232,114,...,[],0,0,[],0,0,[],0,0,[]
1,0,NORMALIDADE,2023-02-12 18:38:45,1.0,Catete,Rua do Catete,228 - 139,-22.926423,-43.176842,99,...,[],0,0,[],0,0,[],0,0,[]
2,0,NORMALIDADE,2023-02-12 18:38:45,2.0,Copacabana,Rua Tonelero,236 - 9,-22.966793,-43.185999,43,...,[],0,0,[],0,0,[],0,0,[]
3,0,NORMALIDADE,2023-02-12 18:38:45,3.0,Ipanema,Avenida Epitácio Pessoa,1910 - 1602,-22.979735,-43.202540,36,...,[],0,0,[],0,0,[],0,0,[]
4,0,NORMALIDADE,2023-02-12 18:38:45,4.0,Barra da Tijuca,Avenida Ministro Ivan Lins,1770 - 11,-23.012736,-43.298339,36,...,[],0,0,[],0,0,[],0,0,[]


In [29]:
polygons.status()['status_name'].value_counts()

NORMALIDADE    79
Name: status_name, dtype: int64

#### Update url to monitor open events at given time

In [5]:
polygons.urls[0]['url'] = api_root + '/comando/bq/open' + '?at=2023-01-08 21:00:00'

display(polygons.status().head())

,status_code,status_name,timestamp,cluster_id,main_neighborhood,main_route,main_street_number_range,lat_centroid,lng_centroid,label_count,...,alagamento_enchente_ids,enchente_count,enchente_status,enchente_ids,vazamento_count,vazamento_status,vazamento_ids,sirene_count,sirene_status,sirene_ids
0,0,NORMALIDADE,2023-02-12 15:59:38,0,Barra da Tijuca,Avenida Armando Lombardi,3098 - 67,-23.006631,-43.310232,114,...,[],0,0,[],0,0,[],0,0,[]
1,0,NORMALIDADE,2023-02-12 15:59:38,1,Catete,Rua do Catete,228 - 139,-22.926423,-43.176842,99,...,[],0,0,[],0,0,[],0,0,[]
2,0,NORMALIDADE,2023-02-12 15:59:38,2,Copacabana,Rua Tonelero,236 - 9,-22.966793,-43.185999,43,...,[],0,0,[],0,0,[],0,0,[]
3,0,NORMALIDADE,2023-02-12 15:59:38,3,Ipanema,Avenida Epitácio Pessoa,1910 - 1602,-22.979735,-43.202540,36,...,[],0,0,[],0,0,[],0,0,[]
4,0,NORMALIDADE,2023-02-12 15:59:38,4,Barra da Tijuca,Avenida Ministro Ivan Lins,1770 - 11,-23.012736,-43.298339,36,...,[],0,0,[],0,0,[],0,0,[]


### Retrive monitoring history by looping thorugh time interval at given frequency

### Polygons monitoring endpoints testing

In [2]:
root = 'http://127.0.0.1:5000'
# root = 'https://bolsao-api-j2fefywbia-rj.a.run.app'

urls = [
    'city/history/window?start=2023-02-07 00:00:00&end=2023-02-08 12:00:00',
    'city/history/open?at=2023-02-07 21:30:00',
    'polygons/history/window?start=2023-02-07 00:00:00&end=2023-02-08 12:00:00',
    'polygons/history/open?at=2023-02-07 21:30:00',
    'polygons/overview/history/window?start=2023-02-07 00:00:00&end=2023-02-08 12:00:00',
    'polygons/overview/history/open?at=2023-02-07 21:30:00',
    'ipp/polygons/history/window?start=2023-02-07 00:00:00&end=2023-02-08 12:00:00',
    'ipp/polygons/history/open?at=2023-02-07 21:30:00',
    'ipp/polygons/overview/history/window?start=2023-02-07 00:00:00&end=2023-02-08 12:00:00',
    'ipp/polygons/overview/history/open?at=2023-02-07 21:30:00',
]

In [3]:
import requests

fail = []
for url in urls:
    try: requests.get(f'{root}/{url}').json()
    except: fail.append(url)
        
print('Endpoints status:')
if len(fail) == 0: print('- All good.')
else: print('Failure:'); print(fail)

Endpoints status:
All good.
